# Imports/setup

In [ ]:
import ee
import os
import pandas as pd
import geopandas as gpd
# import json
import geemap
import time

import os
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import get_ceo_url, whisp_stats_shapefile_and_ceo_url
from parameters.config_output_naming import (
    geo_id_column, 
    out_file_wide, 
    out_shapefile
)
from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    geometry_to_geo_id,
    feature_to_geo_id,
    geo_id_or_ids_to_feature_collection,
    shapefile_to_ee_feature_collection,
    register_feature_and_set_geo_id,
    register_fc_and_set_geo_id,
    register_feature_and_append_to_csv,
    register_fc_and_append_to_csv,
    add_geo_ids_to_feature_col_from_lookup_df,
    add_geo_ids_to_feature_col_from_lookup_csv
)

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid#,
    # get_scale_from_image
)

def whisp_alt_stats_as_df (roi): 
    %store roi
    %run alt_whisp_stats.ipynb   
    return df_out
   

##### Set up asset registry

In [ ]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : with GEOIDS

##### Input: list of Geo ids

In [ ]:
example_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids
#all_geo_ids= example_ids

In [ ]:
#make into a feature collection
roi_geoid = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#### Inputs : without GEOIDS

##### Input: GEE asset (feature collection) 

In [ ]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'
#"projects/fdap-remi/assets/seg_civ_scoopac"
# example_plot_feature_col_asset = "projects/fdap-remi/assets/seg_civ_coopara"
# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0010

In [ ]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)#.limit(10)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

# fc = ee.FeatureCollection(fc.toList(fc.size()).slice(0,3)) #_sample

fc.size()

# df = collection_properties_to_df(collection=fc)#,property_selection=["system:index")

fc = fc.limit(10)

In [ ]:
register_fc_and_append_to_csv(
    feature_col=fc,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    # output_lookup_csv="whisp_output_table.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=True,
    remove_temp_csv=False,
    debug=True)


In [ ]:
join_id_column = "system:index"

fc_new = add_geo_ids_to_feature_col_from_lookup_csv(fc,"temp_geo_id_lookup.csv",join_id_column,geo_id_column,override_checks=True,debug=True)


In [ ]:
collection_properties_to_df(fc_new)

In [ ]:
#run for feature collection (NB needs speeding up...)
roi_asset = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True
)

# task = ee.batch.Export.table.toDrive(collection=roi_asset, 
#                      description="Whisp_roi_asset_w_geo_id", 
#                      folder="Whisp_results",
#                      fileFormat="CSV")

# task.start()

print ("done")

In [ ]:
roi_asset

##### Input: polygon feature collection

In [ ]:
geom_1 =ee.Geometry.Polygon(
        [[[102.19867776108482, 3.0835809888660344],
          [102.1988172359536, 3.082670357801012],
          # [102.20001886559227, 3.082078035157836],
          # [102.2013385124276, 3.081745922272887],
          [102.2014355657636, 3.0825815609469625],
          [102.20011591892828, 3.0836528916202233]]],None, False);

geom_2 = ee.Geometry.Polygon(
        [[[102.18777620396249, 3.079459767638651],
          # [102.18831264576546, 3.0785384196951506],
          [102.1895786484205, 3.0792026473624907],
          [102.18899929127328, 3.08005971470826]]],None, False);

feat_1 = ee.Feature(geom_1)
feat_2 = ee.Feature(geom_2)

In [ ]:
fc = ee.FeatureCollection([feat_1,feat_2])

In [ ]:
#run for feature collection (NB needs speeding up..get geo ids then use map over property setting)
roi_asset = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=False
)


##### Input: point feature collection

In [ ]:
#get test points from polygon feature collection (NB functionality for registering points in asset registry coming soon)
fc.map(get_centroid)

In [ ]:
roi_poly = ee.FeatureCollection(fc.map(get_centroid))

##### Input: shapefile

In [ ]:
# example_shapefile = /path_to/my_shapefile
example_shapefile = out_shapefile

In [ ]:
roi_shapefile = shapefile_to_ee_feature_collection(example_shapefile)

##### Input: drawn feature

In [ ]:
zoom_level =16

Map = geemap.Map(center=[6,-4], zoom=zoom_level)

# Map.add_basemap("Esri.WorldImagery")# optional basemaps from geemap

sc = Sidecar(title='draw polygon')

with sc:display(Map)
print("Draw shape on map before running next cell")

In [ ]:
# #NB User input needed: draw boundary (under 400ha) on the map before running this cell
# try:
#     drawn_feature = (Map.draw_last_feature)
#     if drawnMap.addLayer(drawn_feature)
# except:
#     print("No drawn input. Making feature from map bounds instead") 
#     drawn_feature = ee.Feature(Map.getBounds({'asGEOJSON':True}))
#     Map.addLayer(drawn_feature)
# drawn_feature

In [ ]:
#register geo id and add to feature
feature_w_geo_id = register_feature_and_set_geo_id(
    drawn_feature,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True)

In [ ]:
roi_drawn = feature_w_geo_id

#### Get plot boundaries

In [ ]:
#roi = roi_geoid
# roi = fc_new#roi_asset
#roi = roi_shapefile
#roi = roi_polygon
#roi = roi_points
roi = roi_drawn

### Whisp it

In [ ]:
df = whisp_alt_stats_as_df(roi)

### Display it

In [ ]:
df

#### Export it to CSV file

In [ ]:
#export output csv
df.to_csv(path_or_buf=out_file_wide, header=True)

print ("output csv: ", out_file_wide)

#### Export it to CEO link

Temporary login details available for demo purposes here: modules/config_ceo.py

In [ ]:
ceo_url = whisp_stats_shapefile_and_ceo_url(roi, df, plot_id_column, out_shapefile)

print('CEO URL:', ceo_url) 